In [1]:
bat_scoring_system= {'R': 2, 'B': -1, '4s': 1, '6s':3, 'Duck':-10}
bowl_scoring_system={'O': 12, 'B': 2, 'M': 40,'R':-1, 'W': 25 }

In [2]:
import requests
from bs4 import BeautifulSoup
import math
import time

In [3]:
url= 'https://www.cricbuzz.com/live-cricket-scorecard/22408/kxip-vs-dc-13th-match-indian-premier-league-2019'

In [4]:
response= requests.get(url)


0.9040319919586182

In [5]:
soup= BeautifulSoup(response.content)


/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


-0.06117510795593262

In [6]:
first_soup= soup.findAll(id="innings_1")[0]

In [7]:
def text_list(hst):
    hst=str(hst)
    counter=0
    lis=[]
    w=''
    for let in hst:
        
        
        if let=='<':
            counter+=1
            if len(w.strip())>0:
                lis.append(w)
            w=''
            
        elif let=='>':
            counter=0
            
        elif counter==0:
            w+=let
    return lis
            
        
        

In [8]:
def bowler_scorer(bowler_dict, scoring_system=bowl_scoring_system):
    points=0
    for key in bowl_scoring_system.keys(): 
        points+= bowl_scoring_system[key]*bowler_dict[key]
    if bowler_dict['W']>1:
        points+= 10*(bowler_dict['W']-1)
    return points
    

In [9]:
def bowler_dict_maker(bowl, bowl_headers=['Bowler', 'O', 'M', 'R', 'W'], scoring_system=bowl_scoring_system):
    bowl_headers=bowl_headers
    bowl=str(bowl)
    bowler_dict={}
    raw_url= str(bowl).split('''href="''')[1].split('''"''')[0].split('/')[:-1]
    bowl_text= text_list(bowl)
    url= 'cricbuzz.com/'
    for piece in raw_url:
        if len(piece)>0:
            url+=piece + '/'
    bowler_dict['URL']=url
    ob= bowl_text[1].split('.')
    bowler_dict['O']=int(ob[0])
    try: 
        bowler_dict['B']=int(ob[1])
    except:
        bowler_dict['B']=0
    
    bowler_dict[bowl_headers[0]]= bowl_text[0].split('(')[0].strip()

    for i in range(2, len(bowl_headers)):
        
        try:
            bowler_dict[bowl_headers[i]]=float(bowl_text[i])
        except:
            bowler_dict[bowl_headers[i]]=bowl_text[i]
            
    bowler_dict['Bowling_Points']= bowler_scorer(bowler_dict)
            
    return bowler_dict
    
    

In [10]:
def innings_bowling_dicts(inns, soup=soup, scoring_system=bowl_scoring_system, headers=['Bowler', 'O', 'M', 'R', 'W']):
    innings_dicts={}
    soup= soup.findAll(id="innings_"+str(inns))[0]
    bowlers= soup.findAll(class_="cb-col cb-col-100 cb-ltst-wgt-hdr")[1]
    raw_bowlers= bowlers.findAll(class_="cb-col cb-col-100 cb-scrd-itms ")
    raw_bowlers= list(filter(lambda x:len(str(x))>350, raw_bowlers))
    url=''
    for bowler in raw_bowlers:
        d= bowler_dict_maker(bowler)
        url=d['URL']
        innings_dicts[url]=d
        
    return innings_dicts

In [11]:
header_soup= soup.findAll(id="innings_"+str(1))[0]
header_soup= soup.findAll(class_='cb-col cb-col-100 cb-scrd-sub-hdr cb-bg-gray')[0]
headers= text_list(str(header_soup))

In [12]:
def batting_points(bat_dict, bat_scoring_system= bat_scoring_system):
    points=0
    if bat_dict['R']==0 and bat_dict['Method']!="not out":
        points-= abs(bat_scoring_system['Duck'])
    points+= bat_scoring_system['R']*bat_dict['R'] + bat_scoring_system['B']*bat_dict['B'] 
    points+=bat_scoring_system['4s']*bat_dict['4s'] + bat_scoring_system['6s']*bat_dict['6s']
    runs=bat_dict['R']
    milestone_bonus= 5* math.ceil(((int(.01+runs/25)**2)/2))
    points+= milestone_bonus
    return points

In [13]:
def batsman_dict_maker(bat, headers= headers, bat_scoring_system=bat_scoring_system):
    bat= str(bat)
    batsman_dict={}
    raw_url= str(bat).split('''href="''')[1].split('''"''')[0].split('/')[:-1]
    url= 'cricbuzz.com/'
    for piece in raw_url:
        if len(piece)>0:
            url+=piece + '/'
    text= text_list(str(bat))
    batsman_dict['URL']=url
    batsman_dict['Method']= text[1]
    del text[1]
    for i in range(0, len(text)):
        text[i]=text[i].split('(')[0].strip()
        try:
            batsman_dict[headers[i]]=float(text[i])
        except:
            batsman_dict[headers[i]]=text[i]
    batsman_dict['Batting Points']= batting_points(batsman_dict, bat_scoring_system)
            
    return batsman_dict
    
    
    
            
        


In [14]:
def innings_batting_dicts(inns, soup=soup, scoring_system=bat_scoring_system):
    innings_dicts={}
    soup= soup.findAll(id="innings_"+str(inns))[0]
    header_soup= soup.findAll(class_='cb-col cb-col-100 cb-scrd-sub-hdr cb-bg-gray')[0]
    headers= text_list(str(header_soup))
    batters= soup.findAll(class_="cb-col cb-col-100 cb-ltst-wgt-hdr")[0]
    raw_batsmen= batters.findAll(class_='cb-col cb-col-100 cb-scrd-itms')
    raw_batsmen= list(filter(lambda x:len(str(x))>350, raw_batsmen))
    url=''
    for batsman in raw_batsmen:
        d= batsman_dict_maker(batsman, headers=headers, bat_scoring_system=scoring_system)
        url=d['URL']
        innings_dicts[url]=d
        
    return innings_dicts
    
    

In [15]:
innings_batting_dicts(1)

{'cricbuzz.com/profiles/8733/': {'URL': 'cricbuzz.com/profiles/8733/',
  'Method': 'lbw b Chris Morris',
  'Batsman': 'Lokesh Rahul',
  'R': 15.0,
  'B': 11.0,
  '4s': 2.0,
  '6s': 1.0,
  'SR': 136.36,
  'Batting Points': 24.0},
 'cricbuzz.com/profiles/10420/': {'URL': 'cricbuzz.com/profiles/10420/',
  'Method': 'lbw b Lamichhane',
  'Batsman': 'Sam Curran',
  'R': 20.0,
  'B': 10.0,
  '4s': 3.0,
  '6s': 1.0,
  'SR': 200.0,
  'Batting Points': 36.0},
 'cricbuzz.com/profiles/2195/': {'URL': 'cricbuzz.com/profiles/2195/',
  'Method': 'run out (Dhawan)',
  'Batsman': 'Mayank Agarwal',
  'R': 6.0,
  'B': 9.0,
  '4s': 0.0,
  '6s': 0.0,
  'SR': 66.67,
  'Batting Points': 3.0},
 'cricbuzz.com/profiles/9429/': {'URL': 'cricbuzz.com/profiles/9429/',
  'Method': 'c Pant b Lamichhane',
  'Batsman': 'Sarfaraz Khan',
  'R': 39.0,
  'B': 29.0,
  '4s': 6.0,
  '6s': 0.0,
  'SR': 134.48,
  'Batting Points': 60.0},
 'cricbuzz.com/profiles/6349/': {'URL': 'cricbuzz.com/profiles/6349/',
  'Method': 'c Pan

In [16]:
innings_bowling_dicts(1)

{'cricbuzz.com/profiles/9585/': {'URL': 'cricbuzz.com/profiles/9585/',
  'O': 4,
  'B': 0,
  'Bowler': 'Kagiso Rabada',
  'M': 0.0,
  'R': 32.0,
  'W': 2.0,
  'Bowling_Points': 76.0},
 'cricbuzz.com/profiles/8333/': {'URL': 'cricbuzz.com/profiles/8333/',
  'O': 4,
  'B': 0,
  'Bowler': 'Chris Morris',
  'M': 0.0,
  'R': 30.0,
  'W': 3.0,
  'Bowling_Points': 113.0},
 'cricbuzz.com/profiles/9781/': {'URL': 'cricbuzz.com/profiles/9781/',
  'O': 3,
  'B': 0,
  'Bowler': 'Avesh Khan',
  'M': 0.0,
  'R': 30.0,
  'W': 0.0,
  'Bowling_Points': 6.0},
 'cricbuzz.com/profiles/11158/': {'URL': 'cricbuzz.com/profiles/11158/',
  'O': 4,
  'B': 0,
  'Bowler': 'Sandeep Lamichhane',
  'M': 0.0,
  'R': 27.0,
  'W': 2.0,
  'Bowling_Points': 81.0},
 'cricbuzz.com/profiles/8175/': {'URL': 'cricbuzz.com/profiles/8175/',
  'O': 4,
  'B': 0,
  'Bowler': 'Harshal Patel',
  'M': 0.0,
  'R': 37.0,
  'W': 0.0,
  'Bowling_Points': 11.0},
 'cricbuzz.com/profiles/8424/': {'URL': 'cricbuzz.com/profiles/8424/',
  'O':

In [17]:
innings_batting_dicts(2)

{'cricbuzz.com/profiles/12094/': {'URL': 'cricbuzz.com/profiles/12094/',
  'Method': 'c Rahul b Ashwin',
  'Batsman': 'Prithvi Shaw',
  'R': 0.0,
  'B': 1.0,
  '4s': 0.0,
  '6s': 0.0,
  'SR': 0.0,
  'Batting Points': -11.0},
 'cricbuzz.com/profiles/1446/': {'URL': 'cricbuzz.com/profiles/1446/',
  'Method': 'lbw b Ashwin',
  'Batsman': 'Shikhar Dhawan',
  'R': 30.0,
  'B': 25.0,
  '4s': 4.0,
  '6s': 0.0,
  'SR': 120.0,
  'Batting Points': 44.0},
 'cricbuzz.com/profiles/9428/': {'URL': 'cricbuzz.com/profiles/9428/',
  'Method': ' b Viljoen',
  'Batsman': 'Shreyas Iyer',
  'R': 28.0,
  'B': 22.0,
  '4s': 5.0,
  '6s': 0.0,
  'SR': 127.27,
  'Batting Points': 44.0},
 'cricbuzz.com/profiles/10744/': {'URL': 'cricbuzz.com/profiles/10744/',
  'Method': ' b Shami',
  'Batsman': 'Rishabh Pant',
  'R': 39.0,
  'B': 26.0,
  '4s': 3.0,
  '6s': 2.0,
  'SR': 150.0,
  'Batting Points': 66.0},
 'cricbuzz.com/profiles/7664/': {'URL': 'cricbuzz.com/profiles/7664/',
  'Method': 'c (sub)Nair b Sam Curran',

In [18]:
innings_bowling_dicts(2)

{'cricbuzz.com/profiles/1593/': {'URL': 'cricbuzz.com/profiles/1593/',
  'O': 4,
  'B': 0,
  'Bowler': 'Ravichandran Ashwin',
  'M': 0.0,
  'R': 31.0,
  'W': 2.0,
  'Bowling_Points': 77.0},
 'cricbuzz.com/profiles/10420/': {'URL': 'cricbuzz.com/profiles/10420/',
  'O': 2,
  'B': 2,
  'Bowler': 'Sam Curran',
  'M': 0.0,
  'R': 11.0,
  'W': 4.0,
  'Bowling_Points': 147.0},
 'cricbuzz.com/profiles/7909/': {'URL': 'cricbuzz.com/profiles/7909/',
  'O': 4,
  'B': 0,
  'Bowler': 'Mohammed Shami',
  'M': 0.0,
  'R': 27.0,
  'W': 2.0,
  'Bowling_Points': 81.0},
 'cricbuzz.com/profiles/9493/': {'URL': 'cricbuzz.com/profiles/9493/',
  'O': 3,
  'B': 0,
  'Bowler': 'Hardus Viljoen',
  'M': 0.0,
  'R': 22.0,
  'W': 1.0,
  'Bowling_Points': 39.0},
 'cricbuzz.com/profiles/12071/': {'URL': 'cricbuzz.com/profiles/12071/',
  'O': 4,
  'B': 0,
  'Bowler': 'Mujeeb Ur Rahman',
  'M': 0.0,
  'R': 36.0,
  'W': 0.0,
  'Bowling_Points': 12.0},
 'cricbuzz.com/profiles/8796/': {'URL': 'cricbuzz.com/profiles/8796